In [2]:
import numpy as np
import galois
import random
from py_ecc.bn128 import G1,G2,multiply, add, curve_order, eq, Z1 


#=========================QAP======================================
U=[
    [2, 2, 1, 2],#x^3 + 2x^2 + x + 2
    [0, 0, 0, 0],#0
    [4, 5, 5, 1],#4x^3 + 5x^2 + 5x + 1
    [4, 3, 6, 1],#4x^3 + 3x^2 + 6x + 1
    [3, 0, 0, 4],#3x^3 + 4
    [6, 6, 3, 6]#6x^3 + 6x^2 + 3x + 6
]

V=[
    [2, 6, 3, 3],#2x^3 + 6x^2 + 3x + 3
    [0, 0, 0, 0],#0
    [5, 1, 4, 5],#5x^3 + x^2 + 4x + 5
    [0, 0, 0, 0],#0
    [0, 0, 0, 0],#0
    [0, 0, 0, 0]#0
]

W=[
    [0, 0, 0, 0],#0
    [6, 6, 3, 6],#6x^3 + 6x^2 + 3x + 6
    [0, 0, 0, 0],#0
    [1, 5, 5, 4],#x^3 + 5x^2 + 5x + 4
    [4, 3, 6, 1],#4x^3 + 3x^2 + 6x + 1
    [3, 0, 0, 4]#3x^3 + 4
]

t=[1, 4, 0, 6, 3]

h=[4, 3, 2]

a=[1, 135, 5, 25, 125, 130]

nb_var=len(U[0])
nb_equation=len(U)

print("nb_var: ",nb_var)
print("nb_equation: ",nb_equation)


#curve_order=7 #prenons 7
tau=random.randint(1,curve_order)
print("tau: ",tau)

vectG1=[]
vectG2=[]
for i in range(nb_var):
    vectG1.append(multiply(G1,tau**i))
    vectG2.append(multiply(G2,tau**i))

print("vectG1: ",vectG1)
print("vectG2: ",vectG2)


nb_var:  4
nb_equation:  6
tau:  12738792738280556218909583262447997622899071401584202882349358829652222829373
vectG1:  [(1, 2), (8979490367670550859422561738577510889623218101050154851287257522907833932702, 8011274512265446911401334922557014135645065537145408840875435796562211624108), (14915346715790110777154167714974372028619231771653361284543387465251206967326, 21852479879160873724738503121718668933827848581519354354027189491303680735767), (15160679663994996498817622946881718987760973313046620389998251451850301843346, 12463991366564391588224420540730672493110100180787249982506051231019341038870)]
vectG2:  [((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531)), ((1289663622522273316681772252403328582510143785395732286533916

In [3]:
def eval_polynomial_desc(poly, x):
    result = 0
    # L'ordre des puissances est décroissant, donc on commence avec la puissance la plus élevée
    power = len(poly) - 1
    for coef in poly:
        result += coef * (x ** power)
        power -= 1  # Diminuer la puissance pour l'itération suivante
    return result

In [4]:
#coté du prover
from py_ecc.bn128 import pairing, FQ

#calcul de [A]_1

A1=Z1
for i in range(nb_equation):
    sum_inner=Z1#neutre

    for j in range(nb_var):
        sum_inner=add(sum_inner,multiply(vectG1[j],U[i][j]))
        #print(f"temp {i},{j}: ",sum_inner)

    A1=add(A1,multiply(sum_inner,a[i]))
    #print("A1: ",A1)
    
print("A1: ",A1)
#print("G1:",G1)

#calcul de [B]_2

B2=Z1
for i in range(nb_equation):
    sum_inner=Z1#neutre

    for j in range(nb_var):
        sum_inner=add(sum_inner,multiply(vectG2[j],V[i][j]))
        #print(f"temp {i},{j}: ",sum_inner)

    B2=add(B2,multiply(sum_inner,a[i]))
    #print("B2: ",B2)

print("B2: ",B2)


#calcul de [C]_1

C1_1=Z1
for i in range(nb_equation):
    sum_inner=Z1#neutre

    for j in range(nb_var):
        sum_inner=add(sum_inner,multiply(vectG1[j],W[i][j]))
        #print(f"temp {i},{j}: ",sum_inner)

    C1_1=add(C1_1,multiply(sum_inner,a[i]))
    #print("C1: ",C1)

C1_2=Z1
t_tau=eval_polynomial_desc(t, tau)
for i in range(len(h)):
    C1_2=add(C1_2,multiply(multiply(vectG1[i],t_tau),h[i]))
    #print("C1: ",C1)

#C1=C1_1+C1_2
C1=C1_1


print("C1: ",C1)

print("\nG1:",G1)
print("G2:",G2)

print("les types de A B C G1 G2: ",type(A1[1]),type(B2),type(C1),type(G1),type(G2))

A = multiply(G2, 5)
B = multiply(G1, 6)
C = multiply(G2, 5 * 6)
assert pairing(A, B) == pairing(G2, C), "Pairing check failed!"
# assert pairing(B2, A1) == pairing(G2, C1), "Pairing check failed!"


A1:  (17149719556406448516547282173460244887016340797070736527425348095336755468962, 8763542154283747189095420285116789361939689495099207314829407551990521646369)
B2:  ((15050025292802991116848577447692728303666609685638840440407149452485245069039, 14633208853222754099926102045917306637126155420385209826080041289647171471030), (18101687982580993471545801193473136251607936098197354355604256819955838360044, 15346747968019125620211657723466938241727813924354484397499565010346537232456))
C1:  (2635976507198416430003163201911058437432628641066206550127895268783550388371, 9890957291005120647017984484729241900758622753599821163397281788255611725089)

G1: (1, 2)
G2: ((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))
les types de A

: 